# Optimization-Based Hedging Formulation

## 1. Notation

### 1) Main Portfolio

+ The whole underlying universe $\mathbf{U}_N$ contains $N$ underlying symbols.
+ A main porfolio is given by the notional vector $\mathbf{p}$, which is a $N*1$ vector.
+ $G^\mathbf{p}$ -- Gross of the main porfolio
+ $N^\mathbf{p}$ -- Net of the main portfolio
+ $\sigma_C^{\mathbf{p}}$ -- Common risk of the main portfolio

### 2) Hedging Portfolio

+ Hedging instrument universe $\mathbf{U}^h_H$ contains $H$ hedging instruments, which could overlap with $\mathbf{U_N}$
+ From the risk perspective, the "broken-out" matrix for hedging instruments is $\mathbf{W}$, which is a $N*H$ matrix
+ $\mathbf{x}$ -- hedge portfolio in $\mathbf{U}^h_H$ ($H*1$ hedge notional vector) which will be optimized
+ $\mathbf{h}=\mathbf{W}\mathbf{x}$ -- hedge portfolio in $\mathbf{U}_N$ ($N*1$ "broken-out" hedge notional vector)
+ $V_i$ -- ADV value for hedging instrument $i$

### 3) Optimization Parameters
+ $c$ -- The common risk of the hedged portfolio should be less or equal to $C = c*G^\mathbf{p}$
+ $\delta$ -- The absolute value of the net of the hedge portfolio should be less or equal to $D = \delta*N^\mathbf{p}$
+ $l_i$ -- The gross value of hedging instrument $i$ should be less or equal to $l_i*V_i$

### 4) Portfolio Risk Data

+ Assume there are in total $F$ factors in the risk factor model
+ $\mathbf{X}$ -- $N*F$ factor loading matrix
+ $\mathbf{\Sigma}$ -- $F*F$ factor convariance matrix

## 2. Original Problem

### 1） Utility Fuction

$$\min \sum_i \frac{|x_i|}{V_i}$$

or

$$\min \|\mathbf{P_0}\mathbf{x}\|_1$$

where $\mathbf{P_0}=diag(\frac{1}{V_1}, \frac{1}{V_2},...,\frac{1}{V_H})$

### 2） Constraints

* Common risk constraint:
$$(\mathbf{h}+\mathbf{p})^T\mathbf{X}\mathbf{\Sigma}\mathbf{X}^T(\mathbf{h}+\mathbf{p}) \le C^2$$

or 

$$\mathbf{x}^T\mathbf{P}\mathbf{x}+2\mathbf{q}^T\mathbf{x}+r \le 0 \Longleftrightarrow \|\mathbf{A}^T\mathbf{x}+\mathbf{A}^{-1}\mathbf{q}\|_2 \le (\mathbf{q}^T\mathbf{P^{-1}}\mathbf{q}-r)^{1/2}$$

Please note:
>+ $\mathbf{P}=\mathbf{W}^T\mathbf{X}\mathbf{\Sigma}\mathbf{X}^T\mathbf{W}$ is an $H*H$ matrix
+ $\mathbf{q}=\mathbf{W}^T\mathbf{X}\mathbf{\Sigma}\mathbf{X}^T\mathbf{p}$ is an $H*1$ vector
+ $\mathbf{A}$ is the Cholesky decomposition of $\mathbf{P}$, i.e. $\mathbf{P}=\mathbf{A}\mathbf{A}^T$

> All the above could be pre-calculated and stored. 

>+ $r=(\sigma_C^\mathbf{p})^2-C^2$

* Net constraint:
$$|\sum_i x_i+N_{\mathbf{p}}| \le D$$

or

$$\mathbf{1}^T\mathbf{x} \le (\delta-1)N_{\mathbf{p}}$$
$$-\mathbf{1}^T\mathbf{x} \le (\delta+1)N_{\mathbf{p}}$$


* Liquidity constraints:
$$|x_i| \le l_i * V_i \text{ for any } i$$

or

$$\mathbf{I}\mathbf{x} \le \mathbf{L}\mathbf{V}$$
$$-\mathbf{I}\mathbf{x} \le \mathbf{L}\mathbf{V}$$

where $\mathbf{I}$ is the identity matrix, $\mathbf{L}$ is the diagonal matrix with the liquidity parameters and $\mathbf{V}$ is the ADV vector.

## 3. Optimization Problem after $l_1$-norm handling

Let $\mathbf{y}=\begin{bmatrix}\mathbf{x}\\\mathbf{v}\end{bmatrix}$ be the new optimization vector with dimention $2H*1$.

### 1) Utility Function

$$\min \mathbf{1}^T\mathbf{v}$$

or 

$$\min \begin{bmatrix}\mathbf{0_{H*1}^T} & \mathbf{1_{H*1}^T}\end{bmatrix}\mathbf{y}$$

### 2) Constraints

* $l_1$-norm handling extra constraints:
$$-\mathbf{v} \le \mathbf{P_0}\mathbf{x} \le \mathbf{v}$$

or

$$\begin{bmatrix}\mathbf{P_0} & -\mathbf{I} \\ -\mathbf{P_0} & -\mathbf{I} \end{bmatrix}\mathbf{y} \le \mathbf{0_{2H*1}}$$


* Net constraints:

$$\begin{bmatrix}\mathbf{1}^T_{H*1} & \mathbf{0}^T_{H*1} \\ -\mathbf{1}^T_{H*1} & \mathbf{0}^T_{H*1} \end{bmatrix}\mathbf{y} \le \begin{bmatrix} (\delta -1)N^P \\ (\delta +1)N^P \end{bmatrix}$$

* Liquidity constraints:

$$\begin{bmatrix}\mathbf{I} & \mathbf{0}^T_{H*H} \\ -\mathbf{I} & \mathbf{0}^T_{H*H} \end{bmatrix}\mathbf{y} \le \begin{bmatrix} \mathbf{L}\mathbf{V} \\ \mathbf{L}\mathbf{V} \end{bmatrix}$$

* Common risk constraints:

$$\|\begin{bmatrix} \mathbf{A}^T & \mathbf{0}_{H*H} \end{bmatrix}\mathbf{y}+\mathbf{A}^{-1}\mathbf{q}\|_2 \le (\mathbf{q}^T\mathbf{P^{-1}}\mathbf{q}-r)^{1/2}$$

## 3. Implementation

The above is a typical **SOCP** optimization problem with:

* $\mathbf{c}=\begin{bmatrix}\mathbf{0}_{H*1}\\ \mathbf{1}_{H*1}\end{bmatrix}$
* $\mathbf{G}_0=\begin{bmatrix} \mathbf{P}_0 & -\mathbf{I}  \\ -\mathbf{P}_0 & -\mathbf{I} \\
\mathbf{1}^T_{H*1} & \mathbf{0}^T_{H*1} \\
-\mathbf{1}^T_{H*1} & \mathbf{0}^T_{H*1} \\
\mathbf{I} & \mathbf{0}^T_{H*H} \\ -\mathbf{I} & \mathbf{0}^T_{H*H}\end{bmatrix}, \mathbf{h}_0=\begin{bmatrix} \mathbf{0}^T_{2H*1} \\ (\delta -1)N^P \\ (\delta +1)N^P \\ \mathbf{L}\mathbf{V} \\ \mathbf{L}\mathbf{V} \end{bmatrix}$ 

(in **CVXOPT**, they are denoted as $\mathbf{G}_l$ and $\mathbf{h}_l$ seperately)
* $\mathbf{G}_1=\begin{bmatrix}\mathbf{0}^T_{H*1} & \mathbf{0}^T_{H*1} \\ -\mathbf{A}^T & -\mathbf{0}^T_{H*H} \end{bmatrix}, \mathbf{h}_1=\begin{bmatrix}(\mathbf{q}^T\mathbf{P^{-1}}\mathbf{q}-r)^{1/2} \\ \mathbf{A}^{-1}\mathbf{q}\end{bmatrix}$